https://github.com/eriklindernoren/Keras-GAN/blob/master/dcgan/dcgan.py

Set up environment

Create images folder
Install scipy for use with <code>scipy.misc.imsave()</code> piece.

In [0]:
import os
os.mkdir("images")

!pip install Pillow==5.0.0
!pip install scipy==1.0.1

Import images and save to array

In [0]:
from __future__ import print_function, division

from keras.datasets import mnist
from keras.layers import Input, Dense, Reshape, Flatten, Dropout
from keras.layers import BatchNormalization, Activation, ZeroPadding2D
from keras.layers.advanced_activations import LeakyReLU
from keras.layers.convolutional import UpSampling2D, Conv2D
from keras.models import Sequential, Model
from keras.optimizers import Adam
import scipy.misc
import matplotlib.pyplot as plt
import pandas as pd
import sys
import os
import numpy as np

Move images to pandas

In [0]:
df = pd.read_csv("sample_data/mnist_test.csv")
images = df.iloc[:,1:].to_numpy()
labels = df.iloc[:,0].values.tolist()
print(images.shape)
#images.head(2)
images[1,1:100]

Create images and place into images folder

In [2]:
for i in range(len(images)):
  out = str("images/mnist_" + str(i) + ".png")
  scipy.misc.imsave(out, np.array(images[i]).reshape(28,28))        

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:6: DeprecationWarning: `imsave` is deprecated!
`imsave` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``imageio.imwrite`` instead.
  


Functions to create:

*   the discriminator - <code>build_discriminator</code> 
*   the generator - <code>build_generator</code>
*   an image creator/saver - <code>save_imgs</code>


 

In [0]:
# Discriminator
def build_discriminator():

    model = Sequential()

    model.add(Conv2D(32, kernel_size=3, strides=2, input_shape=img_shape, padding="same"))
    model.add(LeakyReLU(alpha=0.2))
    model.add(Dropout(0.25))
    model.add(Conv2D(64, kernel_size=3, strides=2, padding="same"))
    model.add(ZeroPadding2D(padding=((0,1),(0,1))))
    model.add(BatchNormalization(momentum=0.8))
    model.add(LeakyReLU(alpha=0.2))
    model.add(Dropout(0.25))
    model.add(Conv2D(128, kernel_size=3, strides=2, padding="same"))
    model.add(BatchNormalization(momentum=0.8))
    model.add(LeakyReLU(alpha=0.2))
    model.add(Dropout(0.25))
    model.add(Conv2D(256, kernel_size=3, strides=1, padding="same"))
    model.add(BatchNormalization(momentum=0.8))
    model.add(LeakyReLU(alpha=0.2))
    model.add(Dropout(0.25))
    model.add(Flatten())
    model.add(Dense(1, activation='sigmoid'))

    model.summary()

    img = Input(shape=img_shape)
    validity = model(img)

    return Model(img, validity)

# Generator
def build_generator():

    model = Sequential()

    model.add(Dense(128 * 7 * 7, activation="relu", input_dim=latent_dim))
    model.add(Reshape((7, 7, 128)))
    model.add(UpSampling2D())
    model.add(Conv2D(128, kernel_size=3, padding="same"))
    model.add(BatchNormalization(momentum=0.8))
    model.add(Activation("relu"))
    model.add(UpSampling2D())
    model.add(Conv2D(64, kernel_size=3, padding="same"))
    model.add(BatchNormalization(momentum=0.8))
    model.add(Activation("relu"))
    model.add(Conv2D(channels, kernel_size=3, padding="same"))
    model.add(Activation("tanh"))

    model.summary()

    noise = Input(shape=(latent_dim,))
    img = model(noise)

    return Model(noise, img)

# Image creator/saver
def save_imgs(epoch):
    r, c = 5, 5
    noise = np.random.normal(0, 1, (r * c, latent_dim))
    gen_imgs = generator.predict(noise)

    # Rescale images 0 - 1
    gen_imgs = 0.5 * gen_imgs + 0.5

    fig, axs = plt.subplots(r, c)
    cnt = 0
    for i in range(r):
        for j in range(c):
            axs[i,j].imshow(gen_imgs[cnt, :,:,0], cmap='gray')
            axs[i,j].axis('off')
            cnt += 1
    fig.savefig("images/mnist_%d.png" % epoch)
    plt.close()

In [27]:
# Input shape
img_rows = 28
img_cols = 28
channels = 1
img_shape = (img_rows, img_cols, channels)
latent_dim = 100

optimizer = Adam(0.0002, 0.5)


# Build and compile the discriminator
discriminator = build_discriminator()
discriminator.compile(loss='binary_crossentropy',
    optimizer=optimizer,
    metrics=['accuracy'])

# Build the generator
generator = build_generator()

# The generator takes noise as input and generates imgs
z = Input(shape=(latent_dim,))
img = generator(z)

# For the combined model we will only train the generator
discriminator.trainable = False

# The discriminator takes generated images as input and determines validity
valid = discriminator(img)

# The combined model  (stacked generator and discriminator)
# Trains the generator to fool the discriminator
combined = Model(z, valid)
combined.compile(loss='binary_crossentropy', optimizer=optimizer)


Model: "sequential_7"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_19 (Conv2D)           (None, 14, 14, 32)        320       
_________________________________________________________________
leaky_re_lu_13 (LeakyReLU)   (None, 14, 14, 32)        0         
_________________________________________________________________
dropout_13 (Dropout)         (None, 14, 14, 32)        0         
_________________________________________________________________
conv2d_20 (Conv2D)           (None, 7, 7, 64)          18496     
_________________________________________________________________
zero_padding2d_4 (ZeroPaddin (None, 8, 8, 64)          0         
_________________________________________________________________
batch_normalization_14 (Batc (None, 8, 8, 64)          256       
_________________________________________________________________
leaky_re_lu_14 (LeakyReLU)   (None, 8, 8, 64)         

In [0]:
# Variables
epochs=4000 #4000
batch_size=32
save_interval=50

# Load the dataset
(X_train, _), (_, _) = mnist.load_data()

# Rescale -1 to 1
X_train = X_train / 127.5 - 1.
X_train = np.expand_dims(X_train, axis=3)

# Adversarial ground truths
valid = np.ones((batch_size, 1))
fake = np.zeros((batch_size, 1))


Train to generator object

In [29]:
for epoch in range(epochs):
  print(epoch)
  # ---------------------
  #  Train Discriminator
  # ---------------------

  # Select a random half of images
  idx = np.random.randint(0, X_train.shape[0], batch_size)
  imgs = X_train[idx]

  # Sample noise and generate a batch of new images
  noise = np.random.normal(0, 1, (batch_size, latent_dim))
  gen_imgs = generator.predict(noise)

  # Train the discriminator (real classified as ones and generated as zeros)
  d_loss_real = discriminator.train_on_batch(imgs, valid)
  d_loss_fake = discriminator.train_on_batch(gen_imgs, fake)
  d_loss = 0.5 * np.add(d_loss_real, d_loss_fake)

  # ---------------------
  #  Train Generator
  # ---------------------

  # Train the generator (wants discriminator to mistake images as real)
  g_loss = combined.train_on_batch(noise, valid)

  # Plot the progress
  print ("%d [D loss: %f, acc.: %.2f%%] [G loss: %f]" % (epoch, d_loss[0], 100*d_loss[1], g_loss))

  # If at save interval => save generated image samples
  if epoch % save_interval == 0:
      save_imgs(epoch)

0


/usr/local/lib/python3.6/dist-packages/keras/engine/training.py:493: UserWarning: Discrepancy between trainable weights and collected trainable weights, did you set `model.trainable` without calling `model.compile` after ?
  'Discrepancy between trainable weights and collected trainable'
/usr/local/lib/python3.6/dist-packages/keras/engine/training.py:493: UserWarning: Discrepancy between trainable weights and collected trainable weights, did you set `model.trainable` without calling `model.compile` after ?
  'Discrepancy between trainable weights and collected trainable'


0 [D loss: 1.012166, acc.: 40.62%] [G loss: 0.800072]
1


/usr/local/lib/python3.6/dist-packages/keras/engine/training.py:493: UserWarning: Discrepancy between trainable weights and collected trainable weights, did you set `model.trainable` without calling `model.compile` after ?
  'Discrepancy between trainable weights and collected trainable'


1 [D loss: 0.586359, acc.: 75.00%] [G loss: 1.081539]
2
2 [D loss: 0.580172, acc.: 62.50%] [G loss: 1.130434]
3
3 [D loss: 0.503563, acc.: 71.88%] [G loss: 1.258011]
4
4 [D loss: 0.480252, acc.: 75.00%] [G loss: 1.240510]
5
5 [D loss: 0.378949, acc.: 81.25%] [G loss: 1.457744]
6
6 [D loss: 0.385059, acc.: 79.69%] [G loss: 1.156055]
7
7 [D loss: 0.323712, acc.: 89.06%] [G loss: 0.954327]
8
8 [D loss: 0.432049, acc.: 81.25%] [G loss: 1.069950]
9
9 [D loss: 0.543693, acc.: 73.44%] [G loss: 1.113493]
10
10 [D loss: 0.788098, acc.: 59.38%] [G loss: 2.098939]
11
11 [D loss: 0.707127, acc.: 57.81%] [G loss: 1.805776]
12
12 [D loss: 0.582693, acc.: 71.88%] [G loss: 2.036557]
13
13 [D loss: 0.644626, acc.: 71.88%] [G loss: 2.018442]
14
14 [D loss: 0.744029, acc.: 60.94%] [G loss: 1.787601]
15
15 [D loss: 0.694810, acc.: 62.50%] [G loss: 1.703725]
16
16 [D loss: 0.597588, acc.: 71.88%] [G loss: 1.791701]
17
17 [D loss: 0.620859, acc.: 70.31%] [G loss: 1.717673]
18
18 [D loss: 0.759840, acc.: 64.

View images from first 100 epochs

In [3]:
from IPython.display import Image
Image('images/mnist_100.png')


View images from last epoch

In [4]:
Image('images/mnist_3900.png')